<a href="https://colab.research.google.com/github/LiamAMcNamara/HMMs-Lab3/blob/master/McNamara_Lab_3_HMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CSE 5522 Lab #3: Hidden Markov Models**

Liam McNamara.308

June 11, 2020

---
---

**Part 1 (50 Points):** Implement the Viterbi Algorithm for HMMs for Eisner's Ice Cream Problem (predict whether each day is hot or cold based on the number of ice creams eaten). See elsner.hmm.xls for a bit more discussion on the problem. Remeber that the Viterbi algorithm computes the most likely sequence for an input.

Your solution should be able to handle variable length sequences (in the range of 3-5).

There are 3 files provided for you that have observation/emission probabilities, transition probabilities, and test sequence data (for evaluation), respectively.

Please read the probabilities and observations from their Carmen URLs, *do not hard-code the values*. Refer back to lab 2 for exampes of working with CSV files and how to download them from URLs.

The observation and transition probabilities have rows being the variable of interest, and columns being the conditioning variables. For example, P(2|H) is in the 3rd row (including header), 3rd column (including row label). The columns sum up to 1.

The test data has one line per sequence. When a sequence is less than five observations long, the last  columns are filled with zeros.



Set up the environment

In [72]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Read in the provided data from Carmen URL

In [151]:
emissionURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/observationProbs.csv'
emission_dataframe = pd.read_csv(emissionURL)
transitionURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/transitionProbs.csv'
transition_dataframe = pd.read_csv(transitionURL)
testDataURL = 'http://web.cse.ohio-state.edu/~barker.348/cse5522/lab3/testData.csv'
testData_dataframe = pd.read_csv(testDataURL)

In [167]:

emissionURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/observationProbsLecture.csv'
emission_dataframe = pd.read_csv(emissionURL)
transitionURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/transitionProbsLecture.csv'
transition_dataframe = pd.read_csv(transitionURL)
testDataURL = 'https://raw.githubusercontent.com/LiamAMcNamara/HMMs-Lab3/master/testDataLecture.csv'
testData_dataframe = pd.read_csv(testDataURL)

Print out the top of the dataframes to make sure the data loaded correctly. The data tables should have the following format:

Emission Probabilities [3, 3]: 3 columns (P(x|...), C, H), and 3 rows corresponding the the number of ice creams

Transition Probabilities [3, 4]: 4 columns (P(x|...), C, H, START), and 3 rows (C, H, STOP)

Test Sequence Data [10, 6]: 6 columns (SeqNumber, Obs1...5), and 10 rows corresponing the the sequence sample number.

Note: The test data has one line per sequence. When a sequence is less than five observations long, the last columns are filled with zeros.

In [168]:
print("Emission Probabilities")
display(emission_dataframe.shape)
display(emission_dataframe.head())
print('\n')
print("Transition Probabilities")
display(transition_dataframe.shape)
display(transition_dataframe.head())
print('\n')
print("Test Sequence Data")
display(testData_dataframe.shape)
display(testData_dataframe.head())

Emission Probabilities


(2, 4)

,P(x|...),Sunny,Rainy,Foogy
0,No Umb.,0.9,0.2,0.7
1,Umbrella,0.1,0.8,0.3




Transition Probabilities


(4, 5)

,P(x|...),Sunny,Rainy,Foggy,START
0,Sunny,0.8,0.05,0.15,0.40
1,Rainy,0.2,0.60,0.20,0.32
2,Foggy,0.2,0.30,0.50,0.28
3,STOP,1.0,1.00,1.00,0.00




Test Sequence Data


(2, 6)

,SeqNumber,Obs1,Obs2,Obs3,Obs4,Obs5
0,first,1,1,1,2,2
1,secone,2,1,2,0,0


Initialize matrix variables and shorthand for emission and transition arrays

In [169]:
# In our class notes we referred to the transition matrix as 'A'
# and the emission/observation matrix as 'B'. So I will be doing the same.
A = transition_dataframe.to_numpy()
A = np.delete(A, 0, 1)  # remove extra (label) column included in the dataframe
                        # this column included (C, H, STOP)
B = np.array(emission_dataframe)
B = np.delete(B, 0, 1)  # remove extra (label) column included in the dataframe
                        # this column included (1, 2 , 3)

sequences = testData_dataframe.to_numpy()
sequences = np.delete(sequences, 0, 1)  # remove extra (label) column included in
                                        # the df. This column inc. (1 ... n seqs)

print(A)
print(B)

[[0.8 0.05 0.15 0.4]
 [0.2 0.6 0.2 0.32]
 [0.2 0.3 0.5 0.28]
 [1.0 1.0 1.0 0.0]]
[[0.9 0.2 0.7]
 [0.1 0.8 0.3]]


In [170]:
for n in range(len(sequences)): #len(sequences)
  seq = sequences[n]
  seq = seq[seq != 0]
  print(f'Observed sequence {n+1}: {seq}\n')
  # M.shape = [sequence length, # of hidden variables]
  m_matrix = np.zeros((len(seq), emission_dataframe.shape[1]-1)) 
  # In this case, Sequence of length 3-5 with 2 possible hidden vairable values (H, C)

  # C.shape = [sequence length - 1, # hidden variables]
  c_matrix = np.delete(m_matrix, 0, 0)
  # In this case, Sequence of length 5-1 with 2 hidden variable values
  c_matrix = c_matrix.transpose()
  # print(m_matrix)
  # print(c_matrix)

  seq = seq - 1 #decrement seq for proper indexing
  forward(m_matrix, c_matrix)


Observed sequence 1: [1 1 1 2 2]

Forward phase calculations:
M[0][0]: 0.4 * 0.9
M[0][1]: 0.32 * 0.2
M[0][2]: 0.28 * 0.7
M[2][1]: 0.9 * 0.46451612903225803
M[2][2]: 0.2 * 0.09483870967741935
M[2][3]: 0.7 * 0.15806451612903225
M[3][1]: 0.9 * 0.6106726351749323
M[3][2]: 0.2 * 0.06060784544704911
M[3][3]: 0.7 * 0.11450111909529978
M[4][1]: 0.1 * 0.6849969718658812
M[4][2]: 0.8 * 0.04281231074161757
M[4][3]: 0.3 * 0.1284369322248527
M[5][1]: 0.1 * 0.38787878787878793
M[5][2]: 0.8 * 0.14545454545454545
M[5][3]: 0.3 * 0.13636363636363633
M-Matrix:
[[0.58064516 0.10322581 0.31612903]
 [0.76334079 0.03463305 0.20202615]
 [0.85624621 0.01888455 0.12486924]
 [0.48484848 0.24242424 0.27272727]
 [0.19783617 0.5935085  0.20865533]]
C-Matrix (transposed):
[[1. 3. 3.]
 [1. 3. 1.]
 [1. 1. 1.]
 [1. 2. 3.]]
Most Likely Sequence: [1 1 1 2 2]
___________________________________________________________

Observed sequence 2: [2 1 2]

Forward phase calculations:
M[0][0]: 0.4 * 0.1
M[0][1]: 0.32 * 0.8
M[0][2]

Forward Phase: Calculate values for the M-Matrix. One step at a time. Starting at first state and moving forward. Filling in the C-Marix as we go.

Choose value for last state by maximizing value of hidden state

In [165]:
def forward(m_matrix, c_matrix):
  # use priors to set first row of M matrix
  print("Forward phase calculations:")
  for i in range(B.shape[1]):
    m_matrix[0,i] = A[i,A.shape[1]-1] * B[seq[0], i]
    print(f'M[0][{i}]: {A[i,A.shape[1]-1]} * {B[seq[0], i]}')

  normalizeRow(m_matrix, 0);
  vect = np.zeros(3)
  for s in range(1, len(seq)):
    for k in range(B.shape[1]):
      for j in range(B.shape[1]):
        vect[j] = (A[j][k] * m_matrix[s-1][j])
      max = np.max(vect)
      print(f'M[{s+1}][{k+1}]: {B[seq[s], k]} * {max}')
      m_matrix[s][k] = B[seq[s], k] * max
      c = np.where(vect == max)
      c_matrix[k][s-1]=c[0][0]
    normalizeRow(m_matrix, s)

  c_matrix_print = c_matrix + 1
  print(f'M-Matrix:\n{m_matrix}')
  print(f'C-Matrix (transposed):\n{c_matrix_print.transpose()}')
  # convert C-Matrix to int so we can use its values as indexes
  c_matrix = c_matrix.astype(int)
  backward(m_matrix, c_matrix)

**Backward Phase**: Use final row of M-matrix to find final member of the most likely sequence. From this, us C-Matrix to work backwards and find what the values of the preceding hidden states should be.

In [163]:
def backward(m_matrix, c_matrix):
  MLS = np.zeros_like(seq)
  last_row = m_matrix[len(seq)-1]
  m_max = np.max(last_row)
  last_hidden = np.where(last_row == m_max)
  MLS[len(seq)-1] = last_hidden[0][0]
  #c_matrix = c_matrix.astype(int)
  #print(MLS)
  for i in range(len(seq) - 2, -1, -1):
    #print("i:", i)
    #print(MLS[i+1])
    x = c_matrix[MLS[i+1]][i]
    MLS[i] = x
  MLS_print = MLS + 1
  print(f'Most Likely Sequence: {MLS_print}')
  print('___________________________________________________________\n')

In [171]:
def normalizeRow(m, i):
  # Normalize each row in M-Matrix (each obseervation) to prevent underflow
  norm_m = m  #python uses pass by reference so this will change m
  row = m[i]
  row = row / np.sum(row)
  norm_m[i] = row
